### Previous util functions

In [81]:
from sklearn.metrics.pairwise import cosine_similarity as CS
import similarity

In [86]:
def appendCosineSimilarity (X_train, X_test, embedding, column_name = ''):
    
    cosine_similarity_train = [ CS(embedding[i].reshape(1, -1), embedding[j].reshape(1, -1))[0][0] for i, j in zip(X_train['node'], X_train['target']) ]
    cosine_similarity_test  = [ CS(embedding[i].reshape(1, -1), embedding[j].reshape(1, -1))[0][0] for i, j in zip(X_test['node'], X_test['target']) ]
    
    X_train[column_name] = cosine_similarity_train
    X_test[column_name] = cosine_similarity_test
    
    return X_train, X_test

In [124]:
def appendTS_SS(X_train, X_test, embedding, column_name = '', mult_factor = 1):
    
    ts_ss = similarity.TS_SS()
    ts_score_train = [ ts_ss(embedding[i], embedding[j])*mult_factor for i, j in zip(X_train['node'], X_train['target']) ]
    ts_score_test = [ ts_ss(embedding[i], embedding[j])*mult_factor for i, j in zip(X_test['node'], X_test['target']) ]
    
    X_train[column_name] = ts_score_train
    X_test[column_name] = ts_score_test
    
    X_train = X_train.fillna(value=0)
    X_test = X_test.fillna(value=0)
        
    return X_train, X_test

# 1. Node2Vec 

### Generate Embedding

In [52]:
from gensim.models import Word2Vec
from node2vec import Node2Vec

In [53]:
def n2vecGenerateEmbedding (modelPath, workers = 1):

    
    DG = CreateGraph(X_train, y_train, directed = True)

    # 2.1 Load or Fit model
    try:
        model = Word2Vec.load(modelPath)
    except Exception:
        node2vec = Node2Vec(DG, dimensions=100, walk_length=16, num_walks=100, workers=workers)
        model = node2vec.fit(window=4, min_count=1)
        if savePath:
            model.save(modelPath)
        pass
    
    
    # Save Embeddings
    emb_matrix = [model.wv[str(i)] for i in range(33226)]
    pd.DataFrame(emb_matrix).to_csv("../results/models/node2vec/emb_matrix.csv", sep=',', index=False, header=False)
            

In [54]:
n2vecGenerateEmbedding('../results/models/node2vec/emb_train.model', workers = 12)

### Generate Features

In [112]:
def n2vecGenerateFeatures (X_train, X_test):
    
    
    try:
        n2vec = np.genfromtxt("../results/models/node2vec/emb_matrix.csv", delimiter=',')
    except Exception:
        n2vecGenerateEmbedding('../results/models/node2vec/emb_train.model', workers = 12)
        n2vec = np.genfromtxt("../results/models/node2vec/emb_matrix.csv", delimiter=',')
        pass
    
    X_train, X_test = appendCosineSimilarity(X_train, X_test, embedding=n2vec, column_name='n2vec_cosine_sim')
    X_train, X_test = appendTS_SS(X_train, X_test, embedding=n2vec, column_name='n2vec_ts-ss_sim', mult_factor = 1e-2)
    
    
    return X_train, X_test

In [138]:
X_train, X_test = n2vecGenerateFeatures(X_train, X_test)

# Save Node2Vec Intermediate Results
file_io.SaveData('../data/intermediate/node2vec/', X_train, X_test, y_train, y_test)

In [154]:
# Load Node2Vec Intermediate Data
X_train, X_test, y_train, y_test = file_io.LoadData('../data/intermediate/node2vec/')